[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/quick-tour/namespacing.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/docs/quick-tour/namespacing.ipynb)

# Namespacing with Pinecone

Namespacing is a neat feature in a Pinecone service that allows you to partition your data in an index. When you read from or write to a namespace in an index, you only access data in that particular namespace. In other words, data from two namespaces may have the same ids but different values. Namespacing is useful when you want to reuse the same data processing pipeline but query only a subset of your data.
For example, when you are building a movie recommender system, you could use namespacing to separate recommendations by genre. Another way is to use metadata instead of namespaces for recommendations by genre.

## Prerequisites

A paid Pinecone account as namespaces are not available on the free starter tier.

Install dependencies.

In [1]:
!pip install -qU \
  pinecone-client==3.0.0 \
  pandas==2.0.3

Set up Pinecone.

Before getting started, decide whether to use serverless or pod-based index.

In [ ]:
import os

use_serverless = os.environ.get("USE_SERVERLESS", "False").lower() == "true"

## Creating an Index

Now the data is ready, we can set up our index to store it.

We begin by initializing our connection to Pinecone. To do this we need a [free API key](https://app.pinecone.io).

In [ ]:
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pc.io)
api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'
environment = os.environ.get('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'

# configure client
pc = Pinecone(api_key=api_key)

Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [ ]:
from pinecone import ServerlessSpec, PodSpec

if use_serverless:
    cloud = os.environ.get('PINECONE_CLOUD') or 'PINECONE_CLOUD'
    spec = ServerlessSpec(cloud='aws', region='us-west-2')
else:
    spec = PodSpec(environment=environment)

In [3]:
index_name = "pinecone-namespacing"

In [4]:
import time

# Delete index if exists
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

# Create an index
pc.create_index(
    name=index_name, 
    dimension=2, 
    metric="euclidean",
    spec=spec
)

# wait for index to be ready before connecting
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

pc.list_indexes().names()

['arxiv-test',
 'azure-openai-langchain-intro',
 'llama-2-rag',
 'openai-retrieval-test',
 'pinecone-namespacing']

In [5]:
# Connect to the index
index = pc.Index(index_name)

### Generate movie data

In [6]:
# Generate some data
import pandas as pd

df = pd.DataFrame()
df["id"] = ["Wall-E", "Up", "Ratatouille", "Toy Story"]
df["vector"] = [[1., 1.], [2., 2.], [3., 3.], [4., 4.]]
df

,id,vector
0,Wall-E,"[1.0, 1.0]"
1,Up,"[2.0, 2.0]"
2,Ratatouille,"[3.0, 3.0]"
3,Toy Story,"[4.0, 4.0]"


### Insert vectors without specifying a namespace

In [7]:
# Insert vectors without specifying a namespace
index.upsert(vectors=zip(df.id, df.vector))
index.describe_index_stats()

{'dimension': 2,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 4}},
 'total_vector_count': 4}

### Insert vectors into a namespace

In [8]:
romantic_comedies = ["Wall-E", "Ratatouille"]
romcom_df = df[df.id.isin(romantic_comedies)]
romcom_df

,id,vector
0,Wall-E,"[1.0, 1.0]"
2,Ratatouille,"[3.0, 3.0]"


In [9]:
# Insert vectors into a namespace
index.upsert(vectors=zip(romcom_df.id, romcom_df.vector), namespace="romantic-comedy")
index.describe_index_stats()

{'dimension': 2,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 4},
                'romantic-comedy': {'vector_count': 2}},
 'total_vector_count': 6}

### Query top-3 results, without a namespace

In [10]:
query_results = index.query(vector=df[df.id == "Wall-E"].vector[0], top_k=3)
query_results

{'matches': [{'id': 'Wall-E', 'score': 0.0, 'values': []},
             {'id': 'Up', 'score': 1.99999905, 'values': []},
             {'id': 'Ratatouille', 'score': 7.99999809, 'values': []}],
 'namespace': ''}

### Query top-3 results, with a namespace

We should expect to see only romantic comedies in the query results.

In [11]:
query_results = index.query(vector=
    df[df.id == "Wall-E"].vector[0], top_k=3, namespace="romantic-comedy"
)
query_results

{'matches': [{'id': 'Wall-E', 'score': 0.0, 'values': []},
             {'id': 'Ratatouille', 'score': 7.99999809, 'values': []}],
 'namespace': 'romantic-comedy'}

### Delete the index

In [12]:
# delete the index
pc.delete_index(index_name)